In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

import thesis_helper
functions = thesis_helper.Thesis_Helper()

from flair.embeddings import TransformerWordEmbeddings
from flair.data import Sentence

# init embedding
#embedding = TransformerWordEmbeddings('bert-base-uncased')
embedding = TransformerWordEmbeddings('Ivo/emscad-skill-extraction')

Some weights of the model checkpoint at Ivo/emscad-skill-extraction were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
annotations = '/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Combined/Taxonomy/Normal/Annotated/combined_annotations.csv'

In [3]:
df = pd.read_csv(annotations,sep=';')

#Filling any empty context columns with 'empty'
df['left_context'] = df['left_context'].fillna('empty')
df['right_context'] = df['right_context'].fillna('empty')

df['concatenated'] = df['left_context'] + ' | ' + df['candidate_skill'] + ' | ' + df['right_context']
print('Number of annotated rows ',df.shape[0])

Number of annotated rows  20836


In [4]:
def bert_embedder(text):

    string = Sentence(text)
    embedding.embed(string)

    # Creating a list which stores the indexes of the | symbols
    bar_indexes = []
    #Creating a list which stores the embedding_tensors
    embedding_tensors = []

    #Checking the sentence object for the | symbols and storing their indexes
    for x in range(1,len(string)+1):
        if '|' in str(string.get_token(x)):
            bar_indexes.append(x)

    #Collecting the embeddings for every index between the indexes in bar_indexes
    word_embedding_indexes = range(bar_indexes[0]+1,bar_indexes[1])
    for x in word_embedding_indexes:
        embedding_tensors.append(pd.Series(string[x].embedding))
        embedding_tensors.append(5)

    #Removing last 5 from list
    embedding_tensors.pop()

    #Turning the elements from embedding_tensors into dataframe rows
    row = pd.DataFrame()
    for x in range(0,len(embedding_tensors)):
        row = row.append(pd.DataFrame(pd.Series(embedding_tensors[x])))

    row = row.transpose().reset_index(drop=True)
    
    #Changing the column names in order to make pd.concat work later
    row.columns = [x for x in range(0,len(row.columns))]
    return row

In [5]:
df['embeddings'] = df['concatenated'].progress_apply(bert_embedder)

  3%|▎         | 674/20836 [01:02<31:09, 10.78it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/ivowings/opt/anaconda3/envs/Thesis/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-ecfafd301f1f>", line 1, in <module>
    df['embeddings'] = df['concatenated'].progress_apply(bert_embedder)
  File "/Users/ivowings/opt/anaconda3/envs/Thesis/lib/python3.8/site-packages/tqdm/std.py", line 810, in inner
    return getattr(df, df_function)(wrapper, **kwargs)
  File "/Users/ivowings/opt/anaconda3/envs/Thesis/lib/python3.8/site-packages/pandas/core/series.py", line 4108, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas/_libs/lib.pyx", line 2467, in pandas._libs.lib.map_infer
  File "/Users/ivowings/opt/anaconda3/envs/Thesis/lib/python3.8/site-packages/tqdm/std.py", line 805, in wrapper
    return func(*args, **kwargs)
  File "<ipython-input-4-cccaa89e3ff8>", line 4, in bert_embedder
    embed

TypeError: object of type 'NoneType' has no len()

In [ ]:
x_bert = pd.concat(df['embeddings'].tolist()).reset_index(drop=True)
x_bert = x_bert.fillna(0)
x_bert.head()

In [8]:
%%time
functions.model_performance(x_bert, df['label'])

In [ ]:
df['pos'] = df['candidate_skill'].progress_apply(functions.pos_tagger)
df['pos'] = df['pos'].progress_apply(functions.sequence_counter)

pos_dicts = df[['pos']]
pos_dicts = pos_dicts['pos'].apply(pd.Series)
pos_dicts = pos_dicts.fillna(0).astype(int)

df['dep'] = df['candidate_skill'].progress_apply(functions.dep_tagger)
df['dep'] = df['dep'].progress_apply(functions.sequence_counter)

dep_dicts = df[['dep']]
dep_dicts = dep_dicts['dep'].apply(pd.Series)
dep_dicts = dep_dicts.fillna(0).astype(int)

x_pos = pos_dicts.join(dep_dicts,lsuffix='_gram', rsuffix='_pos')

x = x_bert.join(x_pos, lsuffix='_embedding', rsuffix='_pos')

In [10]:
%%time

functions.model_performance(x.fillna(0), df['label'])

  0%|          | 0/6 [00:00<?, ?it/s]

Starting model evaluation
We are at classifier  LogisticRegression(max_iter=10000000000000000000000, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 33.8min finished
 17%|█▋        | 1/6 [33:47<2:48:56, 2027.34s/it]

We are at classifier  GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 60.6min finished
 33%|███▎      | 2/6 [1:34:26<3:18:20, 2975.22s/it]

...

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 24.3min finished
 83%|████████▎ | 5/6 [2:05:05<20:40, 1240.66s/it]

We are at classifier  MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,),
              max_iter=10000000000000000000000, random_state=456,
              solver='lbfgs')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 19.4min finished
100%|██████████| 6/6 [2:24:28<00:00, 1444.78s/it]


CPU times: user 1.68 s, sys: 2.92 s, total: 4.6 s
Wall time: 2h 24min 29s


,Classifier,Precision,Recall,F1
0,LR,0.801670,0.787471,0.782443
1,GBC,0.764995,0.663556,0.689041
2,SGD,0.809853,0.777329,0.774928
3,RF,0.900056,0.573797,0.620707
4,SVM,0.883234,0.706934,0.744163
5,MLP,0.792364,0.783205,0.777376
